In [3]:
import numpy as np
import copy


import scipy.special
import torch
import torch.nn as nn

In [4]:
# TODO: Convert the notebook to /tests directory
n_ops = 7
n_nodes = 3

alpha_normal = []
alpha_reduce = []

for i in range(n_nodes):
    # create alpha parameters over parallel operations
    alpha_normal.append(nn.Parameter(
        1e-3 * torch.randn(i + 2, n_ops)))
    alpha_reduce.append(nn.Parameter(
        1e-3 * torch.randn(i + 2, n_ops)))
    
    
[i.shape for i in alpha_normal]

[torch.Size([2, 7]), torch.Size([3, 7]), torch.Size([4, 7])]

In [5]:
# Set alphas to zero

In [17]:
alphas = []
alpha_reduce = []
n_ops = 7

class A(nn.Module):
    def __init__(self):
        super(A, self).__init__()
        self.alphas = []
        for i in range(n_nodes):
            # create alpha parameters over parallel operations
            self.alphas.append(nn.Parameter(
                1e-3 * torch.randn(i + 2, n_ops)))
            alpha_reduce.append(nn.Parameter(
                1e-3 * torch.randn(i + 2, n_ops)))

# alphas = [torch.tensor(np.random.normal(0, 0.4, size=(2,7))),
#          torch.tensor(np.random.normal(0, 0.4, size=(3,7))), 
#          torch.tensor(np.random.normal(0, 0.4, size=(4,7)))]


# drop = np.array(weights[i, :]).argsort()[:ops_drop]
num_to_drop = [2, 2, 0]
A = A()
# add stages loop
# TODO: Currently the 0 values are sometimes higher than negative values that might show up
for stage in range(3):
    n_ops_reduce = 5 - num_to_drop[stage]

    for alpha in A.alphas:
        topk, indices = torch.topk(alpha[:, :], n_ops_reduce)

        mask = torch.zeros(len(alpha), n_ops)
        a_min = torch.min(alpha)
        epsilon = 0.0001
        print(alpha)
        print(indices)
#         print(mask)
#         print(indices)
#         print(mask.scatter_(1, indices, True) * alpha)
        alpha.data = mask.scatter_(1, indices, True) * alpha
        alpha.data[alpha.data == 0] = a_min - epsilon
        print(alpha)
        break
    break
        
#     print(alphas, "\n")
        
# A.alphas

Parameter containing:
tensor([[-0.0013,  0.0006,  0.0001,  0.0007, -0.0015,  0.0002,  0.0010],
        [ 0.0008,  0.0005, -0.0027, -0.0006,  0.0006, -0.0011, -0.0011]],
       requires_grad=True)
tensor([[6, 3, 1],
        [0, 4, 1]])
Parameter containing:
tensor([[-0.0028,  0.0006, -0.0028,  0.0007, -0.0028, -0.0028,  0.0010],
        [ 0.0008,  0.0005, -0.0028, -0.0028,  0.0006, -0.0028, -0.0028]],
       requires_grad=True)


In [ ]:
# Test for skip-connections

In [20]:
PRIMITIVES = [
    "max_pool_3x3",
    "avg_pool_3x3",
    "skip_connect",  # identity
    "conv_1x5_5x1",
    "conv_3x3",
    "sep_conv_3x3",
    "dil_conv_3x3"
]

alpha = [torch.tensor(np.random.normal(0, 0.4, size=(2,7))),
         torch.tensor(np.random.normal(0, 0.4, size=(3,7))), 
         torch.tensor(np.random.normal(0, 0.4, size=(4,7)))]

alpha_pairwise = [torch.tensor([1]), 
                  torch.tensor([0.3346, 0., 0.]), 
                  torch.tensor([0.0, 0.0, 0.0, 0.1686, 0.0, 0.0])]

In [22]:
def convert_tensor_alphas(alpha_concat, nodes=3):
    alphas = []
    for a_i in get_edge_indices(nodes):
        alphas.append(
            torch.Tensor(alpha_concat[a_i[0]:a_i[1]]))
        
    # print(alpha_concat, alphas)
    return alphas

def get_edge_indices(nodes=3):
    # Amount of nodes for each edge
    j = [i for i in range(2, nodes+2)] 
    
    prev = 0
    indices = []
    for i in j:
        if prev != 0:
            indices.append((sum(j[:j.index(prev)+1]), 
                            sum(j[:j.index(i)+1])))
        else:
            indices.append((0, sum(j[:j.index(i)+1])))
        prev = i
    return indices

def parse(alpha, k, primitives=PRIMITIVES):
    gene = []
    for edges in alpha:
        # edges: Tensor(n_edges, n_ops)
        edge_max, primitive_indices = torch.topk(
            edges[:, :], 1
        )  # ignore 'none' ##removed none
        topk_edge_values, topk_edge_indices = torch.topk(edge_max.view(-1), k)
        node_gene = []
        for edge_idx in topk_edge_indices:
            prim_idx = primitive_indices[edge_idx]
            prim = primitives[prim_idx]
            node_gene.append((prim, edge_idx.item()))

        gene.append(node_gene)
    return gene


parse(alpha, 2, switches_normal)

NameError: name 'switches_normal' is not defined

In [ ]:
import torch.nn.functional as F


weights_normal = [F.softmax(a).data.cpu().numpy() for a in alpha]
weights_normal = np.concatenate(weights_normal, axis=0)
switches = switches_normal

idxs = np.where(switches[i])[0].tolist()
drop = np.array(weights_normal[i, :]).argsort()[:2]


idxs, drop, weights_normal[i]


In [25]:
get_edge_indices()

[(0, 2), (2, 5), (5, 9)]

In [45]:
alpha

[a.shape for a in alpha]

[torch.Size([2, 7]), torch.Size([3, 7]), torch.Size([4, 7])]

In [48]:
nodes = 3

def get_edge_indices(nodes=3):
    # Amount of nodes for each edge
    j = [i for i in range(2, nodes+2)] 
    
    prev = 0
    indices = []
    for i in j:
        if prev != 0:
            indices.append((sum(j[:j.index(prev)+1]), 
                            sum(j[:j.index(i)+1])))
        else:
            indices.append((0, sum(j[:j.index(i)+1])))
        prev = i
    return indices

def find_indice(row_idx, nodes=3):
    edges = get_edge_indices(nodes)

    for i, (lower, upper) in enumerate(edges):
        if row_idx >= lower and row_idx <= upper:
            # return index
            idx_alpha = 0 if row_idx-lower == 0 else row_idx-lower-1
            return i, idx_alpha
        
# alpha[][]
for i in range(0, 10):
    print(find_indice(i))

(0, 0)
(0, 0)
(0, 1)
(1, 0)
(1, 1)
(1, 2)
(2, 0)
(2, 1)
(2, 2)
(2, 3)


In [58]:

alpha = [torch.tensor(np.random.normal(0, 0.4, size=(2,7))).cuda(),
         torch.tensor(np.random.normal(0, 0.4, size=(3,7))).cuda(), 
         torch.tensor(np.random.normal(0, 0.4, size=(4,7))).cuda()]
alpha[0][0][2] = 1
alpha[0][1][2] = 1
alpha[1][0][2] = 1
alpha[1][1][2] = 1
alpha[2][0][2] = 1
alpha[2][1][2] = 1

print(np.concatenate(
    [a.detach().cpu().numpy() for a in alpha],
    axis=0))

[[-0.54005336  0.44601386  1.          0.16935927  0.61018918 -0.09079537
   0.05649533]
 [ 0.17622196 -0.83845706  1.          0.11027397 -0.00795754  0.30650679
  -0.34431568]
 [-0.86343314  0.22569936  1.          0.4516483  -0.26683228 -0.34520844
   0.58705117]
 [ 0.07984683  0.58052784  1.         -0.42695922  0.59414872  0.73740815
  -0.17766901]
 [ 0.34435377 -0.26419923 -0.41083582 -0.1372797   0.28563996 -0.20738398
   0.63491066]
 [-0.24708575  0.04247829  1.         -0.54031261  0.25097981  0.22914088
   0.19957397]
 [ 0.17441212  0.0447187   1.          0.12966215  0.51792497  0.32531224
  -0.54569785]
 [-0.3830318  -0.46992206  0.09301063  0.83863576  0.30681551 -0.00942102
  -0.26645276]
 [-0.52736485 -0.33109814  0.13805327 -0.8320009   0.27071955  0.12108847
   0.33667816]]


In [25]:
def get_edge_indices(nodes=3):
    # Amount of nodes for each edge
    j = [i for i in range(2, nodes+2)] 
    
    prev = 0
    indices = []
    for i in j:
        if prev != 0:
            indices.append((sum(j[:j.index(prev)+1]), 
                            sum(j[:j.index(i)+1])))
        else:
            indices.append((0, sum(j[:j.index(i)+1])))
        prev = i
    return indices

def find_indice(row_idx, nodes=3):
    edges = get_edge_indices(nodes)

    for i, (lower, upper) in enumerate(edges):
        if row_idx >= lower and row_idx <= upper:
            # return index
            idx_alpha = 0 if row_idx-lower == 0 else row_idx-lower-1
            return i, idx_alpha

def limit_skip_connections_alphas(alpha, primitives, k=2, nodes=3, num_of_skip_connections=2):
    """Mutate the alpha in-place and return the corresponding gene"""
    gene = parse(alpha, k=k)
    num_sk_enabled = sum([1 for edge in gene
                          for op in edge if op[0] == "skip_connect"])
    epsilon = 0.00
    
    if num_sk_enabled < num_of_skip_connections:
        return gene
    else:
        sk_idx = primitives.index("skip_connect")
        alphas_concat = np.concatenate(
            [a.detach().numpy() for a in alpha],
            axis=0)
        sk_alphas = alphas_concat[:, sk_idx-1:sk_idx]

        for i in range(len(sk_alphas)):
            # Pick skip-connection index with lowest alpha value
            row_idx = np.argmin(sk_alphas)
            # Set to inf so we don't pick this again
            sk_alphas[row_idx] = float("inf")
            
            edge_idx, row_idx = find_indice(row_idx)
            print(alpha[edge_idx][row_idx])
            print(torch.min(alpha[edge_idx][row_idx]))

            alpha[edge_idx][row_idx][sk_idx] = torch.min(alpha[edge_idx][row_idx]) - epsilon
            gene = parse(alpha, k=k)
            
            num_sk_enabled = sum([1 for edge in gene
                                  for op in edge if op[0] == "skip_connect"])
            if num_sk_enabled < num_of_skip_connections:
                return gene

limit_skip_connections_alphas(alpha, PRIMITIVES)
print(np.concatenate(
    [a.detach().numpy() for a in alpha],
    axis=0))


[[-0.19708896  0.1437415   0.08133781  0.13216552 -0.66782436  0.01566144
   0.28501196]
 [-0.06607365 -0.39054451  0.16713766  0.43814778  0.11344568 -0.43010951
  -0.49521541]
 [-0.10706372  0.15323234 -0.4360023   0.77714112  0.10937363 -0.68392082
  -0.0880985 ]
 [-0.78684685 -0.36010764  0.29144824  0.0575596   0.01070957  0.19358707
  -0.10091347]
 [-0.09255316  0.56303111  0.62339725 -0.72657522 -0.02643146  0.32926354
   0.12097027]
 [ 0.29802936  0.49010262 -0.40068117 -0.01854009  0.22592499 -0.49414736
   0.15203245]
 [-0.38684428  0.5841493   0.51792072 -0.27329896 -0.19433815  0.55878352
  -0.06745332]
 [ 0.21673819 -0.28628213  0.         -0.87630712  0.15935514  0.16476185
   0.10258931]
 [ 0.30347341 -0.39530993  0.14879027  0.30460576 -0.05406766 -0.41679011
  -0.52559684]]


In [7]:

def reduce_skip_connections(alphas, num_of_skip_connections=2, nodes=3, 
                           k=2, primitives=PRIMITIVES):
    sk_idx = primitives.index("skip_connect")
    alphas = [a.detach().numpy() for a in alphas]
    alpha_concat = np.concatenate(alphas, axis=0)
    
    # Number of skip-connections enabled in switches
    gene = parse(alphas, switches, k=k)
    num_sk_enabled = sum([1 for edge in gene 
                          for op in edge if op[0] == "skip_connect"])
    
    sk_a = alpha_concat[:, sk_idx-1:sk_idx]
    if num_sk_enabled < num_of_skip_connections:
        alphas = convert_tensor_alphas(alpha_concat)
        gene = parse(alphas, switches, k=k)
        return gene
    else:
        it = 0
        while num_sk_enabled > num_of_skip_connections:
            print("########## iteration", it)
            it += 1
            # Pick skip-connection index with lowest alpha 
            # value
            row_idx = np.argmin(sk_a)
            sk_a[row_idx] = float("inf")

        #     # set alphas to -inf to make sure, prevent it from
        #     # being picked. 
            alpha_concat[row_idx][sk_idx] = float("-inf")
            alphas = convert_tensor_alphas(alpha_concat)

            gene = parse(alphas, switches, k=k)
            num_sk_enabled = sum([1 for edge in gene 
                                  for op in edge if op[0] == "skip_connect"])

            if num_sk_enabled <= num_of_skip_connections:
                # return the new switches
                return gene

limit_skip_connections(alpha)

NameError: name 'PRIMITIVES' is not defined

In [46]:

alphas = [torch.tensor(np.random.normal(0, 0.4, size=(2,3))).cuda(),
         torch.tensor(np.random.normal(0, 0.4, size=(3,3))).cuda(), 
         torch.tensor(np.random.normal(0, 0.4, size=(4,3))).cuda()]

alphas = [a.detach().cpu().numpy() for a in alphas]
alpha_concat = np.concatenate(alphas, axis=0)
alpha_concat

array([[ 0.47157174,  0.0242357 ,  0.40998554],
       [-0.2201242 , -0.36017159,  0.26092069],
       [-0.428018  , -0.56124322, -0.90665213],
       [-0.30024917,  1.09211891,  0.39520035],
       [ 0.57640854, -0.12008687,  0.08212192],
       [ 0.17091093, -0.70449669, -0.36777858],
       [-0.44876285,  0.31320018,  0.43844799],
       [-0.52653206, -0.34502978, -0.13791534],
       [-0.06752017,  0.63286921, -0.00976202]])

In [12]:
sk_a = alpha_concat[:, 1:2]

np.argmin(sk_a)

it = 0
while num_sk_enabled > num_of_sk:
    print("########## iteration", it)
    it += 1
    # Pick skip-connection index with lowest alpha 
    # value
    row_idx = np.argmin(sk_a)
    sk_a[row_idx] = float("inf")

#     # set alphas to -inf to make sure, prevent it from
#     # being picked. 
    alpha_concat[row_idx][sk_idx] = float("-inf")
    alphas = convert_tensor_alphas(alpha_concat)

    gene = parse(alphas, switches, k=k)
    num_sk_enabled = sum([1 for edge in gene 
                          for op in edge if op[0] == "skip_connect"])

    if num_sk_enabled <= num_of_sk:
        # return the new switches
        return gene

array([[-0.43544406],
       [-0.26283069],
       [-0.26637197],
       [-0.25242813],
       [-0.02152324],
       [-0.89322286],
       [-0.29785362],
       [ 0.38971684],
       [ 0.14580411]])

In [66]:
C_in = 28
C_out = 28
kernel_size = 3
stride = 1
padding = 2
dilation = 2
C_mid_mult = 1
affine = False

C_mid = 32
cmid = int(C_out * C_mid_mult)
cmid = C_mid if C_mid else cmid

cmid

32

In [67]:

op = nn.Sequential(
    nn.ReLU(inplace=False),
    nn.Conv2d(C_in, C_in, kernel_size, stride,
              padding, dilation, groups=C_in, bias=0),
    nn.Conv2d(C_in, cmid, kernel_size=1,
              padding=0, bias=0),
    nn.BatchNorm2d(cmid, affine=affine),
    nn.ReLU(inplace=False),
    nn.Conv2d(cmid, cmid, kernel_size,
              stride=1, padding=(kernel_size-1)//2,
              dilation=1, groups=cmid, bias=0),
    nn.Conv2d(cmid, C_out, kernel_size=1,
              padding=0, bias=0),
    nn.BatchNorm2d(C_out, affine=affine))


op

Sequential(
  (0): ReLU()
  (1): Conv2d(28, 28, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2), groups=28, bias=False)
  (2): Conv2d(28, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (4): ReLU()
  (5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
  (6): Conv2d(32, 28, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (7): BatchNorm2d(28, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
)

In [73]:
op(torch.zeros((20, 28, 28, 28)))

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 